# Model input connectivity format preparation 

In [1]:
import mne      # toolbox for analyzing and visualizing EEG data
import os       # using operating system dependent functionality (folders)
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import copy     # Can Copy and Deepcopy files so original file is untouched.
from ipywidgets import IntSlider, Output
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from math import nan
from scipy import stats
from mne_connectivity import spectral_connectivity_epochs
from mne_connectivity.viz import plot_sensors_connectivity
import pyvista 
import mayavi
import vtk


import sys
sys.path.insert(0, '../eegyolk') # path to helper functions
import helper_functions as hf # library useful for eeg and erp data cleaning
import initialization_functions #library to import data
import epod_helper

In [2]:
metadata = pd.read_csv('metadata.csv', sep = ',')

In [3]:
metadata

,eeg_file,ParticipantID,test,sex,age_months,dyslexic_parent,Group_AccToParents,path_eeg,path_epoch,path_eventmarkers,epoch_file
0,105a,105,a,f,17,f,1,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,105a_epo.fif
1,107a,107,a,f,16,m,1,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,107a_epo.fif
2,106a,106,a,m,19,f,0,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,106a_epo.fif
3,109a,109,a,m,21,m,0,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,109a_epo.fif
4,110a,110,a,m,17,m,1,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,110a_epo.fif
...,...,...,...,...,...,...,...,...,...,...,...
96,209a,209,a,m,17,f,1,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,209a_epo.fif
97,210a,210,a,m,19,m,0,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,210a_epo.fif
98,202a,202,a,f,16,Nee,0,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,202a_epo.fif
99,201a,201,a,m,18,m,1,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,201a_epo.fif


In [4]:
epochs = initialization_functions.read_filtered_data(metadata)

Checking out file: 105a_epo.fif
Checking out file: 107a_epo.fif
Checking out file: 106a_epo.fif
Checking out file: 109a_epo.fif
Checking out file: 110a_epo.fif
Checking out file: 112a_epo.fif
Checking out file: 111a_epo.fif
Checking out file: 114a_epo.fif
Checking out file: 115a_epo.fif
Checking out file: 117a_epo.fif
Checking out file: 116a_epo.fif
Checking out file: 118a_epo.fif
Checking out file: 119a_epo.fif
Checking out file: 123a_epo.fif
Checking out file: 122a_epo.fif
Checking out file: 124a_epo.fif
Checking out file: 127a_epo.fif
Checking out file: 125a_epo.fif
Checking out file: 126a_epo.fif
Checking out file: 130a_epo.fif
Checking out file: 128a_epo.fif
Checking out file: 129a_epo.fif
Checking out file: 131a_epo.fif
Checking out file: 135a_epo.fif
Checking out file: 133a_epo.fif
Checking out file: 137a_epo.fif
Checking out file: 138a_epo.fif
Checking out file: 139a_epo.fif
Checking out file: 141a_epo.fif
Checking out file: 144a_epo.fif
Checking out file: 143a_epo.fif
Checking

In [5]:
def input_connectivity_prep(epochs, metadata):     
    # loop over epochs
    dfcon = pd.DataFrame()
    
    for i in range(len(metadata)):
        print(f"Calculating connectivity of file {i} \n", end = '')
        # define values for connectivity
        fmin, fmax = 4., 9.
        sfreq = epochs[i].info['sfreq']  # the sampling frequency
        tmin = 0.0  # exclude the baseline period
        
        # calculate connectivity for all epochs per participant
        con = spectral_connectivity_epochs(
            epochs[i], method='pli', mode='multitaper', fmin=fmin, fmax=fmax,
            faverage=True, tmin=tmin, mt_adaptive=False, n_jobs=1)
        
        # transform connectivity to dataframe 
        epochrow = pd.DataFrame(con.get_data())
        # transpose into row and remove all zero connectivity values
        epochrow = epochrow.loc[~(epochrow==0).all(axis=1)].transpose()
        epochrow['eeg_file'] = metadata['eeg_file'][i]
        epochrow['Group_AccToParents'] = metadata['Group_AccToParents'][i]
        # append row to total dataframe
        dfcon = dfcon.append(epochrow)
        
    return dfcon
    

In [ ]:
dfcon = input_connectivity_prep(epochs, metadata)

Calculating connectivity of file 0 
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..0.800s for estimation (1639 points)
    frequencies: 5.0Hz..8.7Hz (4 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing connectivity for

/tmp/ipykernel_1669500/4118856748.py:13: RuntimeWarning: There were no Annotations stored in <EpochsFIF |  2425 events (all good), -0.200195 - 0.799805 sec, baseline off, ~51 kB, data not loaded,
 'GiepM_FS': 129
 'GiepM_S': 357
 'GiepM_D': 119
 'GiepS_FS': 130
 'GiepS_S': 359
 'GiepS_D': 119
 'GopM_FS': 128
 'GopM_S': 360
 'GopM_D': 120
 'GopS_FS': 130
 and 2 more events ...>, so metadata was not modified.
  con = spectral_connectivity_epochs(
/tmp/ipykernel_1669500/4118856748.py:13: RuntimeWarning: fmin=4.000 Hz corresponds to 3.201 < 5 cycles based on the epoch length 0.800 sec, need at least 1.250 sec epochs or fmin=6.248. Spectrum estimate will be unreliable.
  con = spectral_connectivity_epochs(


    computing connectivity for epoch 27
    computing connectivity for epoch 28
    computing connectivity for epoch 29
    computing connectivity for epoch 30
    computing connectivity for epoch 31
    computing connectivity for epoch 32
    computing connectivity for epoch 33
    computing connectivity for epoch 34
    computing connectivity for epoch 35
    computing connectivity for epoch 36
    computing connectivity for epoch 37
    computing connectivity for epoch 38
    computing connectivity for epoch 39
    computing connectivity for epoch 40
    computing connectivity for epoch 41
    computing connectivity for epoch 42
    computing connectivity for epoch 43
    computing connectivity for epoch 44
    computing connectivity for epoch 45
    computing connectivity for epoch 46
    computing connectivity for epoch 47
    computing connectivity for epoch 48
    computing connectivity for epoch 49
    computing connectivity for epoch 50
    computing connectivity for epoch 51


/tmp/ipykernel_1669500/4118856748.py:13: RuntimeWarning: There were no Annotations stored in <EpochsFIF |  2421 events (all good), -0.200195 - 0.799805 sec, baseline off, ~51 kB, data not loaded,
 'GiepM_FS': 127
 'GiepM_S': 356
 'GiepM_D': 119
 'GiepS_FS': 129
 'GiepS_S': 360
 'GiepS_D': 119
 'GopM_FS': 129
 'GopM_S': 360
 'GopM_D': 119
 'GopS_FS': 130
 and 2 more events ...>, so metadata was not modified.
  con = spectral_connectivity_epochs(
/tmp/ipykernel_1669500/4118856748.py:13: RuntimeWarning: fmin=4.000 Hz corresponds to 3.201 < 5 cycles based on the epoch length 0.800 sec, need at least 1.250 sec epochs or fmin=6.248. Spectrum estimate will be unreliable.
  con = spectral_connectivity_epochs(


    computing connectivity for epoch 27
    computing connectivity for epoch 28
    computing connectivity for epoch 29
    computing connectivity for epoch 30
    computing connectivity for epoch 31
    computing connectivity for epoch 32
    computing connectivity for epoch 33
    computing connectivity for epoch 34
    computing connectivity for epoch 35
    computing connectivity for epoch 36
    computing connectivity for epoch 37
    computing connectivity for epoch 38
    computing connectivity for epoch 39
    computing connectivity for epoch 40
    computing connectivity for epoch 41
    computing connectivity for epoch 42
    computing connectivity for epoch 43
    computing connectivity for epoch 44
    computing connectivity for epoch 45
    computing connectivity for epoch 46
    computing connectivity for epoch 47
    computing connectivity for epoch 48
    computing connectivity for epoch 49
    computing connectivity for epoch 50
    computing connectivity for epoch 51


...

    computing connectivity for epoch 27
    computing connectivity for epoch 28
    computing connectivity for epoch 29
    computing connectivity for epoch 30
    computing connectivity for epoch 31
    computing connectivity for epoch 32
    computing connectivity for epoch 33
    computing connectivity for epoch 34
    computing connectivity for epoch 35
    computing connectivity for epoch 36
    computing connectivity for epoch 37
    computing connectivity for epoch 38
    computing connectivity for epoch 39
    computing connectivity for epoch 40
    computing connectivity for epoch 41
    computing connectivity for epoch 42
    computing connectivity for epoch 43
    computing connectivity for epoch 44
    computing connectivity for epoch 45
    computing connectivity for epoch 46
    computing connectivity for epoch 47
    computing connectivity for epoch 48
    computing connectivity for epoch 49
    computing connectivity for epoch 50
    computing connectivity for epoch 51


/tmp/ipykernel_1669500/4118856748.py:13: RuntimeWarning: There were no Annotations stored in <EpochsFIF |  1176 events (all good), -0.200195 - 0.799805 sec, baseline off, ~51 kB, data not loaded,
 'GiepM_FS': 6
 'GiepM_S': 9
 'GiepM_D': 2
 'GiepS_FS': 82
 'GiepS_S': 235
 'GiepS_D': 76
 'GopM_FS': 103
 'GopM_S': 278
 'GopM_D': 89
 'GopS_FS': 53
 and 2 more events ...>, so metadata was not modified.
  con = spectral_connectivity_epochs(
/tmp/ipykernel_1669500/4118856748.py:13: RuntimeWarning: fmin=4.000 Hz corresponds to 3.201 < 5 cycles based on the epoch length 0.800 sec, need at least 1.250 sec epochs or fmin=6.248. Spectrum estimate will be unreliable.
  con = spectral_connectivity_epochs(


    computing connectivity for epoch 27
    computing connectivity for epoch 28
    computing connectivity for epoch 29
    computing connectivity for epoch 30
    computing connectivity for epoch 31
    computing connectivity for epoch 32
    computing connectivity for epoch 33
    computing connectivity for epoch 34
    computing connectivity for epoch 35
    computing connectivity for epoch 36
    computing connectivity for epoch 37
    computing connectivity for epoch 38
    computing connectivity for epoch 39
    computing connectivity for epoch 40
    computing connectivity for epoch 41
    computing connectivity for epoch 42
    computing connectivity for epoch 43
    computing connectivity for epoch 44
    computing connectivity for epoch 45
    computing connectivity for epoch 46
    computing connectivity for epoch 47
    computing connectivity for epoch 48
    computing connectivity for epoch 49
    computing connectivity for epoch 50
    computing connectivity for epoch 51


/tmp/ipykernel_1669500/4118856748.py:13: RuntimeWarning: There were no Annotations stored in <EpochsFIF |  2443 events (all good), -0.200195 - 0.799805 sec, baseline off, ~51 kB, data not loaded,
 'GiepM_FS': 119
 'GiepM_S': 334
 'GiepM_D': 114
 'GiepS_FS': 127
 'GiepS_S': 355
 'GiepS_D': 118
 'GopM_FS': 158
 'GopM_S': 390
 'GopM_D': 139
 'GopS_FS': 124
 and 2 more events ...>, so metadata was not modified.
  con = spectral_connectivity_epochs(
/tmp/ipykernel_1669500/4118856748.py:13: RuntimeWarning: fmin=4.000 Hz corresponds to 3.201 < 5 cycles based on the epoch length 0.800 sec, need at least 1.250 sec epochs or fmin=6.248. Spectrum estimate will be unreliable.
  con = spectral_connectivity_epochs(


    computing connectivity for epoch 27
    computing connectivity for epoch 28
    computing connectivity for epoch 29
    computing connectivity for epoch 30
    computing connectivity for epoch 31
    computing connectivity for epoch 32
    computing connectivity for epoch 33
    computing connectivity for epoch 34
    computing connectivity for epoch 35
    computing connectivity for epoch 36
    computing connectivity for epoch 37
    computing connectivity for epoch 38
    computing connectivity for epoch 39
    computing connectivity for epoch 40
    computing connectivity for epoch 41
    computing connectivity for epoch 42
    computing connectivity for epoch 43
    computing connectivity for epoch 44
    computing connectivity for epoch 45
    computing connectivity for epoch 46
    computing connectivity for epoch 47
    computing connectivity for epoch 48
    computing connectivity for epoch 49
    computing connectivity for epoch 50
    computing connectivity for epoch 51


In [11]:
df_ana = dfcon.groupby('Group_AccToParents').mean().transpose()

In [18]:
df_ana.nlargest(10, 0)

Group_AccToParents,0,1
96,0.206838,0.208696
129,0.204324,0.202192
989,0.194233,0.208139
128,0.192666,0.196380
954,0.192118,0.190882
960,0.192088,0.205251
32,0.188124,0.194258
956,0.182722,0.190133
932,0.174746,0.180771
988,0.173900,0.183296


In [20]:
df_ana.nlargest(10, 1)

Group_AccToParents,0,1
96,0.206838,0.208696
989,0.194233,0.208139
960,0.192088,0.205251
129,0.204324,0.202192
128,0.192666,0.196380
32,0.188124,0.194258
954,0.192118,0.190882
956,0.182722,0.190133
988,0.173900,0.183296
932,0.174746,0.180771
